<p align="center">
    <b><big>CRISP-DM (Cross Industry Standard Process for Data Mining)</big></b>
</p>


<b><font color="green">1. Crips flow</font></b>

<img src="assets/Crips flow.jpg">

<b>2. Phases and generic tasks of CRISP-DM</b>

<img src="assets/phases-and-generic-tasks-of-CRISP-DM.png">

# Bussiness Understanding

Business Understanding adalah tahap awal dari proses analisis data atau proyek data science, dan tujuannya adalah untuk memahami kebutuhan bisnis, tujuan proyek, serta bagaimana hasil dari analisis atau model akan digunakan untuk memberikan nilai tambah pada bisnis

## Ruang lingkup

Dalam suatu perusahaan penyedia jasa asuransi kesehatan sedang mengalami masalah karena banyaknya klaim asuransi dan tidak efisiennya program asuransi dalam beberapa tahun terakhir. Perusahaan membutuhkan studi pengetahuan untuk menjauhkan pelanggan dari penyakit DIABETES. Dengan mengetahui profil pelanggan yang dapat terjangkit penyakit DIABETES, serta mengetahui faktor penyebanya. Akan membantu perusahaan dalam mengurangi claim terhadap asuransi dan memperoleh keuntungan sebanyak mungkin.

## Problem statement

1. Atribut apa yang paling berpengaruh terhadap penyakit DIABETES? <br>
2. Memprediksi pelanggan yang dapat berpotensi menderita DIABETES

## Goal

1. mengetahui atribut apa saja penyebab diabetes? <br>
2. Membuat model yang dapat memprediksi diabetes?

# Data Understanding

    Memahami domain masalah dan tujuan analisis.
    Mengumpulkan data dan memahami sumber data.
    Menentukan label (target) dan fitur (atribut bebas).
    Memahami struktur data dan konteks dari variabel-variabel yang ada.

## Overview data

Dataset ini merupakan bagian dari dataset besar yang dimiliki oleh National Institutes of Diabetes-Digestive-Kidney Diseases di Amerika Serikat. Data tersebut digunakan untuk penelitian diabetes pada wanita Indian Pima berusia 21 tahun ke atas yang tinggal di Phoenix, kota terbesar kelima di negara bagian Arizona, Amerika Serikat. <br>
<b>sumber: https://www.kaggle.com/datasets/mathchi/diabetes-data-set.</b> <br>
<b>Penjelasan Data </b><br>

    Pregnancies: The number of pregnancies
    Glucose: 2-hour plasma glucose concentration in the oral glucose tolerance test
    Blood Pressure: Blood Pressure (Small blood pressure) (mmHg)
    SkinThickness: Skin Thickness (mm)
    Insulin: 2-hour serum insulin (mu U/ml)
    DiabetesPedigreeFunction: A function that calculates the probability of having diabetes according to the descendants
    BMI: Body mass index
    Age: Age (year)
    Outcome: Have the disease (1) or not (0)


<b> Note. Dataset yang akan digunakan sudah dimodifikasi </b>

<b> Load semua library diatas </b>

In [141]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.tree import plot_tree
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

### Load dan overview data

In [142]:
driveawal ='diabetes_modified.csv' #kalau pakai local

In [143]:
df = pd.read_csv(driveawal)


diketahui total data sejumlah 809 dan terdapat 9 kolom (attribut)

### Missing value dan tipe data

In [ ]:
df.info()

diketahui jumlah data 806 dan terdapat beberapa data yang null (missing value) dan tipe data yang digunakan

In [ ]:
# tampilkan jumlah null
null_counts = df.isnull().sum()
print("\nJumlah nilai null di setiap kolom:")
print(null_counts)

### Cek duplikasi

In [ ]:
# Memeriksa duplikat
duplikat = df.duplicated()
# Menampilkan baris yang duplikat
baris_duplikat = df[df.duplicated()]
print("\n Jumlah baris duplikat:\n", len(baris_duplikat))

### Menentukan label (target) dan fitur (atribut bebas)

In [ ]:
X = df.drop(columns=["Outcome"])  # Mengambil semua kolom kecuali kolom "label"
y = df["Outcome"]  # Mengambil kolom "label

X.head() #tampilkan contoh data

## Exploratory Data Analysis (EDA)

Tahap ini melibatkan:

    Menganalisis data untuk memahami pola, distribusi, dan hubungan antar variabel.
    Mendeteksi nilai yang hilang dan outliers.
    Visualisasi data untuk menggali lebih dalam tentang karakteristik dataset.
    Melakukan analisis statistik deskriptif.

In [ ]:
def plots(feature):
    fig = plt.figure(constrained_layout = True, figsize=(10,3))
    gs = gridspec.GridSpec(nrows=1, ncols=4, figure=fig)

    ax1 = fig.add_subplot(gs[0,:3])
    sns.histplot(df.loc[df["Outcome"]==0,feature],
                 kde = False, color = "#004a4d",
                  bins=40,
                 label="Not Diabetes", ax=ax1);
    sns.histplot(df.loc[df["Outcome"]==1,feature],
                 kde = False, color = "#7d0101",
                 bins=40,
                 label="Diabetes", ax=ax1);
    ax2 = fig.add_subplot(gs[0,3])
    sns.boxplot(X[feature], orient="v", color = "#989100",
                width = 0.2, ax=ax2);

    ax1.legend(loc="upper right");


plots("Glucose")

### Pearson corelation matrix

 Korelasi Pearson adalah salah satu metode untuk menghitung korelasi antara dua variabel. Ketika kita menghitung korelasi Pearson untuk setiap pasangan variabel dalam sebuah DataFrame, hasilnya adalah sebuah matriks korelasi.
    Nilai korelasi (r) dekat dengan 1 atau -1 menunjukkan hubungan linear yang kuat.
    Nilai korelasi (r) dekat dengan 0 menunjukkan hubungan linear yang lemah atau tidak ada hubungan linear.

Interpretasi Umum

Berikut adalah interpretasi umum dari nilai korelasi Pearson:

    |r| ≥ 0.9: Korelasi sangat kuat
    0.7 ≤ |r| < 0.9: Korelasi kuat
    0.5 ≤ |r| < 0.7: Korelasi sedang
    0.3 ≤ |r| < 0.5: Korelasi lemah
    |r| < 0.3: Korelasi sangat lemah atau tidak ada korelasi

In [ ]:
# Menghitung matriks korelasi Pearson
correlation_matrix = df.corr(method='pearson')
plt.figure(figsize=(12,10))
# Membuat heatmap dari matriks korelasi
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Matriks Korelasi Pearson')
plt.show()

opsi lain bisa menggunakan scatter plot antar atribut

In [ ]:
# Menentukan threshold
threshold = 0.3

# Mencari pasangan atribut yang berpengaruh berdasarkan threshold
influential_pairs = []

for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) >= threshold:
            influential_pairs.append((correlation_matrix.columns[i], correlation_matrix.columns[j], correlation_matrix.iloc[i, j]))

# Menampilkan hasil
print("Atribut yang saling berpengaruh berdasarkan threshold:")
for pair in influential_pairs:
    print(f"Atribut {pair[0]} dan {pair[1]} memiliki korelasi {pair[2]:.2f}")

print('Pada tahapan ini anda juga bisa membuang atribut yang tidak relevan')

### Mengetahui proporsi jumlah tiap label

In [ ]:
# Menghitung jumlah 0 dan 1
unique, counts = np.unique(y, return_counts=True)
count_dict = dict(zip(unique, counts))
keys_list = list(count_dict.keys())

# Plot jumlah 0 dan 1 ke dalam diagram batang
plt.bar(count_dict.keys(), count_dict.values(), color=['blue', 'green'])
plt.xlabel('Value')
plt.ylabel('Count')
plt.title('Count of Label')
plt.xticks(keys_list, keys_list)
plt.show()

dapat disimpulkan label tidak imbang

### Analisa deskriptif

In [ ]:
df.describe() #jika data categorical "string" bisa gunakan bar plot

dapat dilihat range nilai data dari tiap attribut dan dapat juga untuk melihat outlier

### Cek data outlier (box plot)

In [ ]:
# Membuat boxplot untuk setiap kolom
plt.figure(figsize=(10, 6))
sns.boxplot(data=df.values)
plt.title('Boxplot Semua Kolom')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='Outcome', y='Insulin', data=df)
plt.title('Boxplot Sample Kolom')
plt.show()

# Preprocessing

<img src="assets/Preprocessing.png">

Tahap ini melibatkan:

    Membersihkan data dari nilai-nilai yang hilang dan outliers.
    Transformasi variabel jika diperlukan (misalnya, normalisasi atau standaridasi).
    Encoding variabel kategorikal.
    Membagi dataset menjadi training dan testing set.

## Hapus Missing Value (NULL)

In [ ]:
df_no_null = df.dropna()
df_no_null_reset = df_no_null.reset_index(drop=True)
df_no_null_reset

jumlah data berkurang dari 806 ke 727

## Hapus Duplikat

In [ ]:
df_no_duplicates = df_no_null_reset.drop_duplicates()
df_no_duplicates_reset = df_no_duplicates.reset_index(drop=True)
df_no_duplicates_reset

setelah dihapus duplikasi berkurang dari 727 ke 692

## Hapus outlier (IQR)

IQR adalah singkatan dari Interquartile Range, yaitu rentang antar kuartil. IQR adalah ukuran statistik yang digunakan untuk menggambarkan penyebaran tengah dalam kumpulan data dan sering digunakan untuk mengidentifikasi outlier.

<b>Rumus IQR</b>

IQR adalah selisih antara kuartil ketiga (Q3) dan kuartil pertama (Q1) dalam dataset. Kuartil adalah nilai yang membagi data yang diurutkan menjadi empat bagian yang sama besar.

    Q1 (Kuartil Pertama): Nilai di bawah 25% dari data terletak.
    Q3 (Kuartil Ketiga): Nilai di bawah 75% dari data terletak.
    IQR (Interquartile Range): Dihitung sebagai Q3 - Q1.

In [ ]:
# Mendefinisikan fungsi untuk menghapus outlier menggunakan IQR
def remove_outliers_iqr(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Memfilter data yang berada dalam rentang [lower_bound, upper_bound]
    df_cleaned = df[~((df < lower_bound) | (df > upper_bound)).any(axis=1)]
    return df_cleaned

# Menghapus outlier
df_cleaned = remove_outliers_iqr(df_no_duplicates_reset)

print("Data setelah menghapus outlier:")
df_cleaned

In [ ]:
# Visualisasi data setelah menghapus outliers dengan boxplot
plt.figure(figsize=(10, 6))
sns.boxplot(data=df_cleaned.values)
plt.title('Boxplot Setelah Menghapus Outliers')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='Outcome', y='Insulin', data=df_cleaned)
plt.title('Boxplot Sample Kolom')
plt.show()

## Ubah tipe data

In [160]:
df_cleaned.loc[:, 'Glucose'] = df_cleaned['Glucose'].astype(int)
df_cleaned.loc[:, 'BloodPressure'] = df_cleaned['BloodPressure'].astype(int)

In [ ]:
df_cleaned.describe()

## Cleaning out range.
Bisa dengan menghapus, median, mean

In [ ]:
# Hapus baris yang memiliki nilai 0 di kolom 'SkinThickness' dan 'Insulin'
df_cleaned = df_cleaned[(df_cleaned['SkinThickness'] != 0) & (df_cleaned['Insulin'] != 0)]

# Menampilkan deskripsi dari DataFrame setelah penghapusan
df_cleaned.describe()

In [ ]:
df_cleaned

data berkurang 574 jadi 305

## Susun ulang variable X dan y

In [ ]:
X = df_cleaned.drop(columns=["Outcome"])  # Mengambil semua kolom kecuali kolom "label"
y = df_cleaned["Outcome"]  # Mengambil kolom "label

X.head() #tampilkan contoh data

## Split data

<img src="assets/train_test_split.jpg">

In [ ]:
# Split dataset
# Membagi data menjadi train (70%) dan sementara (20%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Membagi data sementara menjadi validation (50% dari 20% yaitu 10%) dan test (50% dari 20% yaitu 10%)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Memeriksa proporsi data
print(f"Train set: {len(X_train)} samples")
print(f"Validation set: {len(X_val)} samples")
print(f"Test set: {len(X_test)} samples")

## Handeling imbalance data

Apa itu SMOTE?

SMOTE adalah metode yang digunakan untuk menghasilkan sampel-sampel baru dari kelas minoritas (kelas dengan jumlah sampel yang lebih sedikit) untuk menyeimbangkan distribusi kelas dalam dataset. SMOTE bekerja dengan cara membuat sampel-sampel sintetis berdasarkan sampel-sampel yang sudah ada dengan menggunakan pendekatan ketetanggaan.

<img src="assets/SMOTE.png">

In [ ]:
# Function to plot class distribution
def plot_class_distribution(y, title):
    plt.figure(figsize=(8, 6))
    y.value_counts().sort_index().plot(kind='bar', color=['blue', 'green'])
    plt.title(title)
    plt.xlabel('Class')
    plt.ylabel('Frequency')
    plt.xticks(rotation=0)
    plt.show()

# Plot class distribution before SMOTE
plot_class_distribution(y_train, 'Class Distribution Before SMOTE (Train)')

# Apply SMOTE to the training set
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Plot class distribution after SMote
plot_class_distribution(y_train_resampled, 'Class Distribution After SMOTE (Train)')

# Modeling

## Fitur important

In [ ]:
# Membuat model Gradient Boosting
gb = GradientBoostingClassifier()
gb.fit(X_train_resampled, y_train_resampled)

# Menghitung fitur importansi
importances = gb.feature_importances_
indices = np.argsort(importances)

# Plot fitur importansi
plt.figure(figsize=(10, 6))
plt.barh(range(X_train_resampled.shape[1]), importances[indices], align='center')
plt.yticks(range(X_train_resampled.shape[1]), [X_train_resampled.columns[i] for i in indices])
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Feature Importance Plot')
plt.show()

## Training

ANN (Artificial Neural Network) adalah model komputasi yang terinspirasi oleh jaringan saraf biologis, seperti otak manusia. ANN digunakan dalam pembelajaran mesin dan kecerdasan buatan untuk memecahkan berbagai masalah seperti klasifikasi, regresi, dan pengenalan pola. ANN juga merupakan cikal bakal arsitektur deeplearning
<img src="assets/training.png">

In [ ]:
# Membuat model neural network sederhana
tf.keras.backend.clear_session()
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64,  input_shape=(8,), activation='relu'),
    tf.keras.layers.BatchNormalization(),  # Layer Normalization untuk normalisasi fitur
    tf.keras.layers.Dense(64,  activation='relu'),
    tf.keras.layers.BatchNormalization(),  # Layer Normalization untuk normalisasi fitur
    tf.keras.layers.Dense(32,  activation='relu'),
    tf.keras.layers.BatchNormalization(),  # Layer Normalization untuk normalisasi fitur
    tf.keras.layers.Dense(1, activation='sigmoid')
])

opt = tf.keras.optimizers.Adam(learning_rate=0.0001)

# Kompilasi model
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
plot_model(model, to_file=driveawal+'cnn_model.png', show_shapes=True, show_layer_names=False, rankdir="LR")

In [ ]:
# Melatih model
# Membuat callback ModelCheckpoint untuk menyimpan model terbaik berdasarkan akurasi validasi
checkpoint = ModelCheckpoint(
    filepath=driveawal+'best_model_fix.keras',    # Nama file untuk menyimpan model
    monitor='val_loss',          # Metrik yang dipantau
    save_best_only=True         # Hanya menyimpan model terbaik
)

# Membuat callback EarlyStopping untuk menghentikan pelatihan jika tidak ada peningkatan
early_stopping = EarlyStopping(
    monitor='val_loss',          # Metrik yang dipantau
    restore_best_weights = True,
    patience=50       # Jumlah epoch tanpa peningkatan sebelum pelatihan berhenti
)

history = model.fit(X_train_resampled, y_train_resampled, epochs=500, batch_size=32, validation_data=(X_val,y_val), callbacks=[checkpoint, early_stopping])

In [ ]:
# Memplot loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Memplot akurasi
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# <img src="assets/undervitting overvitting.png">

## Testing

In [ ]:
# Evaluasi model dengan data uji
y_pred_encoded = model.predict(X_test)
y_pred = (y_pred_encoded > 0.5).astype(np.int64)

bisa juga menggunakan kfold agar model lebih baik

# Evaluation

## Confussion Matrix

In [ ]:
# Membuat confusion matrix
conf_mat = confusion_matrix(y_test, y_pred)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_mat, annot=True, cmap='Blues', fmt='g')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

## Classification Report

In [ ]:
# Membuat classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

    - Akurasi mengukur seberapa sering model klasifikasi benar dalam memprediksi kelas secara keseluruhan.
    - Presisi mengukur seberapa sering prediksi positif model benar saat sebenarnya positif.
    - Recall mengukur seberapa sering model dapat memprediksi positif dengan benar dari semua kasus yang sebenarnya positif.
    - F1 score adalah ukuran gabungan dari presisi dan recall, yang merupakan harmonic mean dari kedua metrik tersebut.

## Simpan model

In [ ]:
# Simpan model ke format TFLite
best = model.save(driveawal+"model.h5")

# Deployment

## Convert model

In [ ]:
# Konversi model H5 ke format TFLite
model = tf.keras.models.load_model(driveawal+"best_model_fix.keras")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Simpan model TFLite ke file
with open(driveawal+"model2.tflite", "wb") as f:
    f.write(tflite_model)

## Build Simple HTML

<img src="https://miro.medium.com/v2/resize:fit:1400/1*XQjVnZhDL-hvNoe84byyiw.png">